In [1]:
from IPython.display import display, clear_output
from ipywidgets import interact, interactive, Text, Button, GridspecLayout, Layout, Label, VBox, HBox, AppLayout, HTML, Accordion, Dropdown, Output

import pandas as pd
from game_client import GameClient
from serving_client import ServingClient

In [2]:
SERVING_CLIENT = ServingClient()
GAME_CLIENT = GameClient()

## GUI

In [3]:
# section to download the model
workspace_text_in = Text(description="Workspace")
model_text_in = Text(description="Model")
version_text_in = Text(description="Version")
download_model_button = Button(description="Download model", button_style="info")

model_panel = VBox(
    [
        workspace_text_in,
        model_text_in,
        version_text_in,
        download_model_button,
    ],
    layout=Layout(display='flex', flex_flow='column', align_items='flex-end')
)

In [4]:
def download_btn(btn):
    with out:
        clear_output()
        SERVING_CLIENT.download_registry_model(
            workspace=workspace_text_in.value,
            model=model_text_in.value,
            version=version_text_in.value
        )
        print(f"Downloading model: {model_text_in.value} v{version_text_in.value}")

download_model_button.on_click(download_btn)

In [5]:
# section to ping the game
ping_button = Button(description="Query game", button_style="info")
game_id_input = Text(description="Game ID")

game_panel = VBox(
    [
        game_id_input,
        ping_button,
    ],
    layout=Layout(display='flex', flex_flow='column', align_items='flex-end')
)

In [10]:
def gui_output(client):
    print("#"*40)
    print("STATUS")
    print(f"Period: {client.period}\t   Time left: {str(client.time_left)[-5:]}")
    print("#"*40)
    print("TEAMS")
    print(f"Home: {client.home_abbrev}\t | Away: {client.away_abbrev}")
    print(f"{client.home_team}\t | {client.away_team}")
    print("#"*40)
    print("EXPECTED GOAL")
    print(f"{client.home_abbrev} xG: \t | {client.away_abbrev} xG: ")
    print("#"*40)
    print(f"last ping: {client.last_ping_at}")
    print("\n\n")
  

def ping_btn(btn):
    with out:
        clear_output()
        previous_df_length = GAME_CLIENT.features_df.shape[0]
        GAME_CLIENT.ping_game(game_id_input.value)
        #SERVING_CLIENT.predict(GAME_CLIENT.features_df.iloc[previous_df_length:])
        gui_output(GAME_CLIENT)
        
ping_button.on_click(ping_btn)

In [7]:
GAME_CLIENT.ping_game(2021020492)

C:\Users\TJ\miniconda3\envs\nice_play\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [11]:
out = Output()

dashboard = HBox([
    VBox([
        HTML(value="<h2>Menu</h2>"),
        model_panel,
        game_panel
    ]),
    VBox([out], layout=Layout(display="flex", flex_flow="row", align_items="center"))
])

In [12]:
display(dashboard)